#  Neural Network model 

The code here represents the Neural Network machine learning model to predict whether a person is affected by covid or pneumonia or whether the person is in normal state using the chest X-Ray. The Chest X-Ray images have been preproccessed and then the features have been extracted and stored in .mat files. We now load these files and create the Neural Network model.

In [1]:
# Importing the required packages
from utils import *

import os
import pywt
import cv2
import glob
import itertools
import numpy as np
import seaborn as sn
import pandas as pd
import scipy.io as sio
import skimage.io as io
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.svm import SVC
from tensorflow import keras
from skimage import exposure
from scipy.stats import skew
from scipy.stats import kurtosis
from sklearn import preprocessing
from IPython.display import Image
from skimage.color import rgb2gray
from keras.utils import to_categorical
from skimage.feature import greycomatrix
from keras.callbacks import EarlyStopping
from sklearn.metrics import roc_curve, auc
from sklearn.decomposition import KernelPCA
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from skimage.transform import  rescale,resize
from sklearn.preprocessing import MinMaxScaler
from skimage.util import img_as_uint,img_as_ubyte
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
# Mentioning the working directory
source_dir='./'

In [3]:
# Accessing covid.mat file and getting the data from the file
covid_features=sio.loadmat(os.path.join(source_dir,'covid.mat')) 
covid_features=covid_features['covid'] 

# Accessing normal.mat file and getting the data from the file
normal_features=sio.loadmat(os.path.join(source_dir,'normal.mat')) 
normal_features=normal_features['normal']  

# Accessing pneumonia.mat file and getting the data from the file
pneumonia_features=sio.loadmat(os.path.join(source_dir,'pneumonia.mat')) 
pneumonia_features=pneumonia_features['pneumonia']  

In [4]:
# Extracting the scores-i.e the inputs and storing it in X
X = np.concatenate((covid_features[:,:-1],normal_features[:,:-1],pneumonia_features[:,:-1]), axis=0)

# Extracting the target labels, the last column alone
y = np.concatenate((covid_features[:,-1],normal_features[:,-1],pneumonia_features[:,-1]), axis=0)

In a neural network, the outputs of the nodes in one layer are used as the inputs for the nodes in the next layer. Therefore, the activation function determines the range of the inputs to the nodes in the following layer.

In [5]:
# Normalization of the data between 0 and 1
min_max_scaler=MinMaxScaler()
X = min_max_scaler.fit_transform(X)

PCA is a linear method. That is it can only be applied to datasets which are linearly separable. It does an excellent job for datasets, which are linearly separable. But, if we use it to non-linear datasets, we might get a result which may not be the optimal dimensionality reduction. Kernel PCA uses a kernel function to project dataset into a higher dimensional feature space, where it is linearly separable. 

In [6]:
# We use Kernel to reduce the feature components to 64 for the input data
transformer = KernelPCA(n_components = 64, kernel='linear')
X = transformer.fit_transform(X)

In [7]:
# We do the splitting of data set and set the training data to be 80% ,i.e, test data = 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

# From that 80%, test data fraction is set as 25% to get a better output as there is more randomness in the dataset for testing
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state = 1)

In [8]:
# Printing the size for each of the data

print("Size of train data:", np.shape(X_train))
print("Size of train label:", np.shape(y_train))
print("Size of test data:", np.shape(X_test))
print("Size of test label:", np.shape(y_test))

Size of train data: (252, 64)
Size of train label: (252,)
Size of test data: (84, 64)
Size of test label: (84,)


Multi-layer Perceptron (MLP) is a supervised learning algorithm that learns a function. by training on a dataset.
Given a set of x1, x2, ... xN features with target y it can learn a non-linear function approximator for classifciation.
This is used because there can be non linear hidden layers in the middle.

In [9]:
# Creating the Neural Network with hidden layer size as 5 x 2
# lbfgs is an optimiser
# alpha = Regularisation term
clf = MLPClassifier(solver = 'lbfgs', alpha = 1e-5, hidden_layer_sizes = (5, 2), random_state=1)

# Training the dataset
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(5, 2), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [10]:
# Predicting/ testing the model
y_pred = clf.predict(X_test)

#print(y_pred)
#Obtaining the evaluation metrics for the model

conf_mat = confusion_matrix(y_test, y_pred) # Confusion matrix
accuracy = accuracy_score(y_test, y_pred) # Accuracy of prediction

precision = precision_score(y_test, y_pred, average='macro') # Precision
recall = recall_score(y_test, y_pred, average='macro') # Recall
f1 = f1_score(y_test, y_pred, average='macro') # F1-score

C:\Users\iamta\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\iamta\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
print("The Accuracy of the model is: " + str(accuracy))

The Accuracy of the model is: 0.2976190476190476


In [12]:
print("The Precision of the model is: " + str(precision))

The Precision of the model is: 0.0992063492063492


In [13]:
print("The Recall of the model is: " + str(recall))

The Recall of the model is: 0.3333333333333333


In [14]:
print("The F1-Score for the model is: " + str(f1))

The F1-Score for the model is: 0.1529051987767584


In [15]:
print("The Confusion matrix is: ")
conf_mat

The Confusion matrix is: 


array([[25,  0,  0],
       [31,  0,  0],
       [28,  0,  0]], dtype=int64)